<a href="https://colab.research.google.com/github/shreyasseshadri/SC-Project/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras

In [0]:
import numpy as np
import random as rn
rn.seed(123)
np.random.seed(123)
from keras.models import Model
from keras.layers import Input, Dense,Conv2D ,LSTM,Lambda,Flatten,Dropout
from keras.utils import np_utils
from keras import backend as K
tf_session = K.get_session()
from keras import regularizers
import tensorflow as tf
tf.set_random_seed(123)

In [0]:
def stack_dim(X):
    conv_outputs=tf.unstack(X,axis=3)
    return tf.concat(conv_outputs,axis=1)

def get_shape(input_shape):
    assert len(input_shape)==4
    return (input_shape[0],input_shape[1]*input_shape[3],input_shape[2])

def model(input_shape,hidden_size,dense_layer1,dense_layer2):
    X_input=Input(input_shape)
    X=Conv2D(32, (5, 5), input_shape=input_shape, activation='relu')(X_input)
    X=Dropout(0.5)(X)
    X=Lambda(stack_dim,output_shape=get_shape)(X)
    X=LSTM(hidden_size,return_sequences=True)(X)
    X=Dense(dense_layer1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(X)
    X=Flatten()(X)
    X=Dropout(0.5)(X)
    X=Dense(dense_layer2, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='sent_classifier')
    return model

In [0]:
X_train=np.load('drive/My Drive/SC-Project/train_data.npy')

In [0]:
X_test=np.load('drive/My Drive/SC-Project/X_test1.npy')

In [0]:
y_train=np.load('drive/My Drive/SC-Project/y_train.npy')
y_test=np.load('drive/My Drive/SC-Project/y_test1.npy')

In [0]:
m=model(X_train.shape[1:],128,50,2)

In [0]:
from keras.optimizers import SGD,Adam
# opt=SGD(lr=0.05, momentum=0.01,decay=0.0, nesterov=False)
m.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
def train(l,h):
    train_size=h-l
    train_pos=np.where(y_train[l:h][...,1]==1)[0]+l
    train_neg=np.where(y_train[l:h][...,0]==1)[0]+l
    np.random.seed(123)
    pos=np.random.choice(train_pos,train_size//2)
    np.random.seed(123)
    neg=np.random.choice(train_neg,train_size-train_size//2)
    indices=np.append(pos,neg,0)
    np.random.seed(123)
    np.random.shuffle(indices)
    return indices

In [0]:
train_index=train(0,2210)
val_index=train(4000,8000)

In [15]:
val_index

array([4323, 4685, 6487, ..., 7127, 7990, 7329])

In [0]:
from sklearn.utils import resample
ups_train_index = resample(train_index,replace=True,n_samples=4000,random_state=123)

In [18]:
len(ups_train_index)

4000

In [19]:
hist=m.fit(X_train[ups_train_index],y_train[ups_train_index],batch_size=64,epochs=10,verbose=1,validation_data=(X_train[val_index],y_train[val_index]))

Train on 4000 samples, validate on 4000 samples
Epoch 1/10
4000/4000 [==============================] - 1083s 271ms/step - loss: 1.4032 - acc: 0.5842 - val_loss: 1.3593 - val_acc: 0.6162
Epoch 2/10
4000/4000 [==============================] - 1077s 269ms/step - loss: 1.2944 - acc: 0.6940 - val_loss: 1.3520 - val_acc: 0.5982
Epoch 3/10
4000/4000 [==============================] - 1072s 268ms/step - loss: 1.2365 - acc: 0.7305 - val_loss: 1.2817 - val_acc: 0.6597
Epoch 4/10
4000/4000 [==============================] - 1070s 268ms/step - loss: 1.1682 - acc: 0.7722 - val_loss: 1.2486 - val_acc: 0.6853
Epoch 5/10
4000/4000 [==============================] - 1072s 268ms/step - loss: 1.1147 - acc: 0.7903 - val_loss: 1.2090 - val_acc: 0.6945
Epoch 6/10
4000/4000 [==============================] - 1077s 269ms/step - loss: 1.0608 - acc: 0.8097 - val_loss: 1.1858 - val_acc: 0.7095
Epoch 7/10
4000/4000 [==============================] - 1076s 269ms/step - loss: 1.0120 - acc: 0.8333 - val_loss: 1.16

In [20]:
hist=m.fit(X_train[ups_train_index],y_train[ups_train_index],batch_size=64,epochs=10,verbose=1,validation_data=(X_train[val_index],y_train[val_index]))

Train on 4000 samples, validate on 4000 samples
Epoch 1/10
4000/4000 [==============================] - 1080s 270ms/step - loss: 0.8731 - acc: 0.8775 - val_loss: 1.1302 - val_acc: 0.7097
Epoch 2/10
4000/4000 [==============================] - 1080s 270ms/step - loss: 0.8163 - acc: 0.9055 - val_loss: 1.1497 - val_acc: 0.7067
Epoch 3/10
4000/4000 [==============================] - 1078s 269ms/step - loss: 0.8201 - acc: 0.8930 - val_loss: 1.1509 - val_acc: 0.6995
Epoch 4/10
4000/4000 [==============================] - 1081s 270ms/step - loss: 0.7899 - acc: 0.9020 - val_loss: 1.1170 - val_acc: 0.7133
Epoch 5/10
4000/4000 [==============================] - 1077s 269ms/step - loss: 0.7421 - acc: 0.9167 - val_loss: 1.1640 - val_acc: 0.6973
Epoch 6/10
4000/4000 [==============================] - 1075s 269ms/step - loss: 0.7213 - acc: 0.9223 - val_loss: 1.1039 - val_acc: 0.7190
Epoch 7/10
4000/4000 [==============================] - 1074s 269ms/step - loss: 0.7035 - acc: 0.9293 - val_loss: 1.12

In [27]:
len(test_index)

18000

In [0]:
m.save('drive/My Drive/SC-Project/final.h5')

In [0]:
from keras.models import load_model
m = load_model('drive/My Drive/SC-Project/final.h5',custom_objects={"tf": tf})

In [27]:
pred=m.evaluate(X_test,y_test)
print(pred)

25000/25000 [==============================] - 3600s 144ms/step
[0.7009602817535401, 0.90712]
